In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.6 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 101 kB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 20.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 7.8 MB/s 


In [3]:
!pip3 install mvlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.0 MB/s 
     |████████████████████████████████| 38.1 MB 8.9 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [5]:
import os
os.chdir('drive/MyDrive/William_2022/DATASETS/main_trans_ZuCo_sent')

In [28]:
!python3 fusion_cossim_ds.py

[ Epoch 0 ]
fusion_cossim_ds.py:63: RuntimeWarning: invalid value encountered in long_scalars
  acc_SP = sum([cm[i, i] for i in range(1, class_num)]) / total_pred[1: class_num].sum()
fusion_cossim_ds.py:64: RuntimeWarning: invalid value encountered in long_scalars
  pre_i = [cm[i, i] / total_pred[i] for i in range(class_num)]
[0.2542372881355932, nan, nan]
[1.0, 0.0, 0.0]
acc_SP is : nan
pre_i is : [0.2542372881355932, 0.0, 0.0]
rec_i is : [1.0, 0.0, 0.0]
F1_i is : [0.4054054054054054, 0.0, 0.0]
    - [Info] The checkpoint file has been updated.
  - (Training)  loss:  0.64605, accuracy: 29.293 %, elapse: 0.003 min
train_cm: [[36 45 27]
 [36 31 28]
 [30 44 20]]
  - (Validation)  loss:  0.28681, accuracy: 25.424 %, elapse: 0.003 min
valid_cm: [[15  0  0]
 [19  0  0]
 [25  0  0]]
[ Epoch 1 ]
fusion_cossim_ds.py:63: RuntimeWarning: invalid value encountered in long_scalars
  acc_SP = sum([cm[i, i] for i in range(1, class_num)]) / total_pred[1: class_num].sum()
fusion_cossim_ds.py:64: Runti

In [22]:
!python3 eval.py

<Figure size 640x480 with 1 Axes>
test_cm: [[19  0  0]
 [ 0 25  0]
 [ 0  0 15]]
[1.0, 1.0, 1.0]
[1.0, 1.0, 1.0]
acc_SP is : 1.0
pre_i is : 1.0
rec_i is : 1.0
F1_i is : 1.0
test_acc is : 1.0


In [ ]:
def get_sinusoid_encoding_table(n_position, d_hid, padding_idx=None):

    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_hid)

    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2]) 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2]) 

    if padding_idx is not None:

        sinusoid_table[padding_idx] = 0.

    return torch.FloatTensor(sinusoid_table)

In [15]:
import torch
import torch.nn as nn

input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output = -(torch.sum(output))
output1 = torch.sum(output)
print(-output)
print(output1)

tensor(-0.6979)
tensor(0.6979)


In [ ]:
input1

tensor([[-0.3582,  0.9591, -0.2615,  ..., -0.0063, -1.7747,  0.1692],
        [ 0.3798,  0.7602, -0.9672,  ...,  0.5947,  0.2297, -2.5566],
        [ 0.0665,  0.0914,  0.8730,  ..., -0.3564, -1.2019,  0.4453],
        ...,
        [-0.2895, -0.3217, -0.1021,  ...,  0.1548, -1.0092,  0.3306],
        [ 0.6578,  1.8276,  0.5509,  ...,  0.3603, -1.1290, -0.7089],
        [ 0.9944, -0.3286, -1.7661,  ..., -0.9905, -0.2016, -1.0287]])

In [ ]:
import numpy as np
import torch
import torch.nn as nn

n_position = 32 + 1
d_hid = 16
padding_index = 0

x = get_sinusoid_encoding_table(n_position = n_position, d_hid = d_hid, padding_idx= padding_index)

In [ ]:
position_enc = nn.Embedding.from_pretrained(
            get_sinusoid_encoding_table(n_position, 16, padding_idx=0),
            freeze=True)

In [ ]:
import scipy.stats as stats

position = stats.zscore(x, axis=0, nan_policy='omit')

In [ ]:
x

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  3.1098e-01,  9.5042e-01,  9.9833e-02,
          9.9500e-01,  3.1618e-02,  9.9950e-01,  9.9998e-03,  9.9995e-01,
          3.1623e-03,  9.9999e-01,  1.0000e-03,  1.0000e+00,  3.1623e-04,
          1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  5.9113e-01,  8.0658e-01,  1.9867e-01,
          9.8007e-01,  6.3203e-02,  9.9800e-01,  1.9999e-02,  9.9980e-01,
          6.3245e-03,  9.9998e-01,  2.0000e-03,  1.0000e+00,  6.3246e-04,
          1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  8.1265e-01,  5.8275e-01,  2.9552e-01,
          9.5534e-01,  9.4726e-02,  9.9550e-01,  2.9996e-02,  9.9955e-01,
          9.4867e-03,  9.9995e-01,  3.0000e-03,  1.0000e+00,  9.4868e-04,
          1.0000e+00],
        [-7.5680e-01

In [ ]:
import pandas as pd
import numpy as np
import torch
embedding = nn.Embedding.from_pretrained(x)


In [ ]:
embedding

Embedding(33, 16)

In [ ]:
ZAB = pd.read_csv('preprocessed_eeg/ZAB_mean.csv')

In [ ]:
X = ZAB.iloc[: ,3:]
y = ZAB[['sentiment']]

In [ ]:
import scipy.stats as stats

X = X.values

X = stats.zscore(X, nan_policy='omit')  


In [ ]:
X

array([[-1.04371209, -0.55295338, -0.02975346, ..., -1.27011643,
        -1.24163433,  1.143572  ],
       [-0.97055519, -0.34373355, -0.42873807, ..., -1.17468286,
        -1.20170034,  1.10858185],
       [-1.07188663, -0.47707203, -0.1419729 , ..., -1.43705086,
        -1.04091085,  1.10522825],
       ...,
       [ 1.89520183,  0.96557457, -0.35875865, ...,  1.64323915,
         1.68529772, -0.02889971],
       [ 0.81041918,  0.41360321,  0.23197849, ...,  2.41918538,
         2.42521238, -0.30269705],
       [ 1.34901165,  0.76892547, -0.19154129, ...,  0.72621093,
         0.59914903,  0.17134104]])

In [ ]:
df

,sentiment,0,1,2,3,4,5,6,7,8,...,95.7,96.7,97.7,98.7,99.7,100.7,101.7,102.7,103.7,104.7
0,1,0.115292,0.143879,0.334241,0.434646,0.520795,0.468173,0.306373,0.346440,0.466853,...,0.229387,0.178027,0.144655,0.320253,0.174269,0.690055,0.188504,0.169423,0.237246,0
1,2,0.149551,0.166466,0.387147,0.342261,0.410666,0.474177,0.353018,0.332580,0.488462,...,0.223091,0.191049,0.170279,0.293045,0.161509,0.820038,0.204374,0.197841,0.246464,0
2,0,0.128100,0.146247,0.364835,0.419477,0.474424,0.402994,0.292123,0.278302,0.432163,...,0.260286,0.213303,0.110620,0.396562,0.147582,0.609134,0.173841,0.133698,0.289041,0
3,2,0.120657,0.124477,0.278609,0.401074,0.350653,0.401724,0.303856,0.384240,0.623583,...,0.253996,0.217986,0.154110,0.352517,0.167664,0.704545,0.187002,0.182403,0.341818,0
4,1,0.114699,0.126094,0.314186,0.473659,0.532056,0.473613,0.282160,0.379398,0.662826,...,0.340485,0.292042,0.131178,0.400218,0.125483,0.811744,0.209772,0.138863,0.357171,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2,1.165944,1.080790,1.085385,0.925674,0.839938,0.385502,0.631342,0.812552,0.897377,...,0.913871,1.221471,1.231353,1.098190,0.540824,0.275063,0.962136,1.320621,1.186124,0
396,0,1.369036,1.311738,1.164263,0.912532,0.733433,0.351036,0.827040,0.957696,0.941271,...,1.002765,1.233011,1.173364,1.179934,0.565974,0.219899,1.105031,1.391419,1.293336,0
397,2,1.453818,1.283306,1.102204,0.693336,0.587966,0.347930,0.623828,0.647208,0.707732,...,1.141842,1.442878,1.214217,1.054010,0.717485,0.346375,1.177364,1.335392,1.211883,0
398,0,0.997968,1.052413,1.015506,0.907788,0.770344,0.445978,0.839517,0.890748,0.925847,...,1.153078,1.350271,1.307005,1.394599,0.663207,0.255053,1.248587,1.632364,1.447959,0


In [ ]:
df = df.loc[(df != 0).any(axis=1)]


In [ ]:
df = df.reset_index()

In [ ]:
df

,index,sentiment,0,1,2,3,4,5,6,7,...,95.7,96.7,97.7,98.7,99.7,100.7,101.7,102.7,103.7,104.7
0,0,1,0.115292,0.143879,0.334241,0.434646,0.520795,0.468173,0.306373,0.346440,...,0.229387,0.178027,0.144655,0.320253,0.174269,0.690055,0.188504,0.169423,0.237246,0
1,1,2,0.149551,0.166466,0.387147,0.342261,0.410666,0.474177,0.353018,0.332580,...,0.223091,0.191049,0.170279,0.293045,0.161509,0.820038,0.204374,0.197841,0.246464,0
2,2,0,0.128100,0.146247,0.364835,0.419477,0.474424,0.402994,0.292123,0.278302,...,0.260286,0.213303,0.110620,0.396562,0.147582,0.609134,0.173841,0.133698,0.289041,0
3,3,2,0.120657,0.124477,0.278609,0.401074,0.350653,0.401724,0.303856,0.384240,...,0.253996,0.217986,0.154110,0.352517,0.167664,0.704545,0.187002,0.182403,0.341818,0
4,4,1,0.114699,0.126094,0.314186,0.473659,0.532056,0.473613,0.282160,0.379398,...,0.340485,0.292042,0.131178,0.400218,0.125483,0.811744,0.209772,0.138863,0.357171,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,395,2,1.165944,1.080790,1.085385,0.925674,0.839938,0.385502,0.631342,0.812552,...,0.913871,1.221471,1.231353,1.098190,0.540824,0.275063,0.962136,1.320621,1.186124,0
393,396,0,1.369036,1.311738,1.164263,0.912532,0.733433,0.351036,0.827040,0.957696,...,1.002765,1.233011,1.173364,1.179934,0.565974,0.219899,1.105031,1.391419,1.293336,0
394,397,2,1.453818,1.283306,1.102204,0.693336,0.587966,0.347930,0.623828,0.647208,...,1.141842,1.442878,1.214217,1.054010,0.717485,0.346375,1.177364,1.335392,1.211883,0
395,398,0,0.997968,1.052413,1.015506,0.907788,0.770344,0.445978,0.839517,0.890748,...,1.153078,1.350271,1.307005,1.394599,0.663207,0.255053,1.248587,1.632364,1.447959,0


In [ ]:
print("printing column name where infinity is present")
col_name = df.columns.to_series()[np.isinf(df).any()]
print(col_name)
  
# printing row index with infinity
print()
print("printing row index with infinity ")
  
r = df.index[np.isinf(df).any(1)]
print(r)

printing column name where infinity is present
Series([], dtype: object)

printing row index with infinity 
Int64Index([], dtype='int64')


In [ ]:
df_train_eeg = ZAB.iloc[:, 3:]

In [ ]:
df = pd.concat([y, df_train_eeg], axis=1)

In [ ]:
ZAB.values

array([[0, '1',
        'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.',
        ..., 0.237246335, 0, 22],
       [1, '2',
        'Beautifully crafted, engaging filmmaking that should attract upscale audiences hungry for quality and a nostalgic, twisty yarn that will keep them guessing.',
        ..., 0.246464431, 0, 22],
       [2, '0',
        'Bread, My Sweet has so many flaws it would be easy for critics to shred it.',
        ..., 0.289040804, 0, 16],
       ...,
       [397, '2',
        "It's not a particularly good film, but neither is it a monsterous one.",
        ..., 1.211882591, 0, 13],
       [398, '0',
        "Men in Black II achieves ultimate insignificance -- it's the sci-fi comedy spectacle as Whiffle-Ball epic.",
        ..., 1.447958827, 0, 16],
       [399, '2', 'Martyr gets royally screwed and comes back for more.',
        ..., 0.916012228, 0, 9]], dtype=object)

In [ ]:
ZAB['token_count'] = ZAB.new_words.str.replace(',','').str.split().str.len()
token = ZAB.filter(['new_words','token_count'])

In [ ]:
token['token_count'].mean()

17.8225